In [1]:
# base modules
import os
import sys
import copy

# for manipulating data
import numpy as np
import pandas as pd
import math
import dill
import gc

# for Machine Learning
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree

# for visualization
from IPython.display import display
from matplotlib import pyplot as plt

In [2]:
# path to repo
path_to_repo = os.path.dirname(os.getcwd())
sys.path.insert(0, os.path.join(path_to_repo, 'src'))

path_to_repo

'/Users/jiaokan/Workshop/Machine Learning in Practice'

In [3]:
# custom module
from mlpcourse.utils import *

In [4]:
# number of CPUs on my machine:
os.cpu_count()

8

### Load and preprocess data

We'll work with the huge data set of the Corporation Favorita Grocery Sales Forecasting kaggle competition from 2018.

Data avaiable [here](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data)

In [5]:
# types = {'id': 'int64',
#          'int_nbr': 'int32',
#          'store_nbr': 'int8',
#          'unit_sales': 'float32',
#          'onpromotion': 'object'}  # boolean with missing values

In [ ]:
# df_all = pd.read_csv('../data/train.csv',  parse_dates=['date'], dtype=types)
# path_to_dataset = os.path.join(
#     path_to_repo, "data", 'raw_groceries_full')

# df_all = pd.read_feather(path_to_dataset)

# df_chunks = pd.read_csv('../data/train.csv',
#                         parse_dates=['date'], dtype=types, chunksize=100000000)
# df_all = pd.concat(df_chunks)

In [8]:
# df_all.onpromotion.fillna(False, inplace=True)
# df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
# df_all.onpromotion = df_all.onpromotion.astype(bool)

# %time df_all.to_feather('raw_groceries')

In [ ]:
# %time df_all.describe(include='all')

In [10]:
# df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))
# cutoff neg values of unit_sales, take log + 1

In [ ]:
# %%time
# add_datepart(df_all, 'date')

In [ ]:
# df_all

In [23]:
# df_all.to_feather('raw_groceries')

In [ ]:
# items = pd.read_csv('items.csv')
# items.shape

In [ ]:
# items.head()

In [ ]:
# items.dtypes

In [ ]:
# items.item_nbr.unique().shape

In [ ]:
# items.family.unique()

In [17]:
# train_cats(items)

In [ ]:
# items.family.cat.codes.unique()

In [19]:
# items.family = items.family.cat.codes

In [20]:
# items.to_feather('items')

In [ ]:
# itms = df_all['item_nbr'].to_numpy()
# itms.shape

In [ ]:
# itms[:10]

In [23]:
# fam = dict(zip(items.item_nbr, items.family))
# cls = dict(zip(items.item_nbr, items['class']))
# per = dict(zip(items.item_nbr, items.perishable))

In [25]:
# def apply_dict_to_array(conversion_dict, input_array):
#     k = np.array(list(conversion_dict.keys()))
#     v = np.array(list(conversion_dict.values()))

#     mapping_ar = np.zeros(k.max()+1, dtype=v.dtype)
#     mapping_ar[k] = v
#     return mapping_ar[input_array]

In [26]:
# fam_col = apply_dict_to_array(fam, itms)
# cls_col = apply_dict_to_array(cls, itms)
# per_col = apply_dict_to_array(per, itms)

In [27]:
# df_all['family'] = fam_col
# df_all['class'] = cls_col
# df_all['perishable'] = per_col

In [13]:
# df_all.to_feather('raw_groceries')

In [40]:
# df_all = pd.read_feather('raw_groceries')

In [ ]:
# df_all.head()

In [29]:
# stores = pd.read_csv('stores.csv')

In [ ]:
# stores.shape

In [ ]:
# stores.dtypes

In [ ]:
# stores.head()

In [33]:
# train_cats(stores)

In [34]:
# stores.city = stores.city.cat.codes
# stores.state = stores.state.cat.codes
# stores.type = stores.type.cat.codes

In [35]:
# city = dict(zip(stores.store_nbr, stores.city))
# state = dict(zip(stores.store_nbr, stores.state))
# type = dict(zip(stores.store_nbr, stores.type))

In [ ]:
# stores_in_db = df_all['store_nbr'].to_numpy()
# stores_in_db.shape

In [37]:
# city_col = apply_dict_to_array(city, stores_in_db)
# state_col = apply_dict_to_array(state, stores_in_db)
# type_col = apply_dict_to_array(tpe, stores_in_db)

In [38]:
# df_all['city'] = city_col
# df_all['state'] = state_col
# df_all['type'] = tpe_col

In [59]:
# df_all.to_feather('raw_groceries')

In [ ]:
# df_all = pd.read_feather('raw_groceries')
# df_all.head()

features considered by kaggle winners:
- store x item
- store x class
- ...

plus a fine feature engineering (including computing means of products prices over various time windows)

In [40]:
# df_all['store_nbr x item_nr'] = df_all['store_nbr'] * df_all['item_nbr']
# df_all['store_nbr x class'] = df_all['store_nbr'] * df_all['class']

In [ ]:
# df_all.head()

In [44]:
# df_all.to_feather('raw_groceries_full')

### Fit

In [53]:
# df_all = pd.read_feather('raw_groceries_full')

In [ ]:
# df_all.shape

Kaggle first rankers used only 2017 data...

In [55]:
# df = df_all[df_all['Year'] == 2017].copy()

In [ ]:
df.shape

In [57]:
del df_all

In [ ]:
gc.collect()

In [59]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [ ]:
n_valid = 3370464
n_trn = len(df) - n_valid
train, valid = split_vals(df, n_trn)
train.shape, valid.shape

In [15]:
def nwrmsle(y_true, y_pred, w):
    """x_per bool array with value 1 if item is perishable, 0 else"""
    return math.sqrt((w*((y_true-y_pred)**2)).sum() / w.sum())
# def rmse(y_true, y_pred):
#     return math.sqrt(((y_true - y_pred)**2).mean())

In [16]:
def print_score(m, X_train, y_train, X_valid, y_valid):
    w_trn = 1 + .25*X_train[:, -1]  # extract `perishable` column
    w_val = 1 + .25*X_valid[:, -1]  # extract `perishable` column
    print('NWRMSLE on train set: {:.4f}'.format(
        nwrmsle(m.predict(X_train), y_train, w_trn)))
    print('NWRMSLE on valid set: {:.4f}'.format(
        nwrmsle(m.predict(X_valid), y_valid, w_val)))
    print('R^2 on train set: {:.4f}'.format(m.score(X_train, y_train)))
    print('R^2 on valid set: {:.4f}'.format(m.score(X_valid, y_valid)))
    if hasattr(m, 'oob_score_'):
        print('R^2 on oob set: {:.4f}'.format(m.oob_score_))
    return

#### Train on 2017 data

NB: do NOT use OOB_score (would be slow to evaluate performance of a tree outside the small subset it was trained on)

In [ ]:
train.shape

In [ ]:
%%time
trn, y, _ = proc_df(train, 'unit_sales')

In [ ]:
%%time
val, y_val, _ = proc_df(valid, 'unit_sales')

In [66]:
model = RandomForestRegressor(n_jobs=-1,
                              n_estimators=10,
                              max_samples=10_000)

In [ ]:
%prun model.fit(trn,y)

We see that sklearn spends a lot of time converting our input df (trn) into a numpy.ndarray, so let's convert it before calling the fit method:

In [ ]:
%time x = np.array(trn, dtype=np.float32)

In [ ]:
%time x_val = np.array(val, dtype=np.float32)

Let's run lots of trees on small subsets (with `max_samples` set small):

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=200,
                          # max_depth=10,
                          min_samples_leaf=100,
                          max_features=0.75,
                          max_samples=20_000)
%time m.fit(x,y)

In [ ]:
print_score(m, x, y, x_val, y_val)

In [ ]:
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=500,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=50_000)
%time m.fit(x,y)

In [ ]:
print_score(m, x, y, x_val, y_val)

In [ ]:
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=50,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=1_000_000)
%time m.fit(x,y)

In [ ]:
print_score(m, x, y, x_val, y_val)
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=10,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=5_000_000)
%time m.fit(x,y)

In [ ]:
print_score(m, x, y, x_val, y_val)
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=5,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=10_000_000)
%time m.fit(x,y)

In [ ]:
print_score(m, x, y, x_val, y_val)
gc.collect()

In [ ]:
x.shape

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=5,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=0.7)
%time m.fit(x,y)

In [ ]:
print_score(m, x, y, x_val, y_val)
gc.collect()

and we'd be in top 1000...

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=5,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=0.8)
%time m.fit(x,y)

In [ ]:
print_score(m, x, y, x_val, y_val)
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=10,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=0.6)
%time m.fit(x,y)

In [ ]:
print_score(m, x, y, x_val, y_val)
gc.collect()

#### Train and validate on kaggle winner's data

See winner's solution [here](https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting/discussion/47582)

train: 20170614 - 20170719 (id=118867503 to id=122667528)

valid: 20170726 - 20170810 (id=123296175 to id=124975583)

In [6]:
df = pd.read_feather('raw_groceries_full')

In [7]:
df_train = df[(df['id'] >= 118867503) & (df['id'] < 122667528)]
df_test = df[(df['id'] >= 123296175) & (df['id'] < 124975583)]
del df

In [ ]:
df_train.head()

In [ ]:
df_train.shape, df_test.shape

In [9]:
trn, y, _ = proc_df(df_train, 'unit_sales')
val, y_val, _ = proc_df(df_test, 'unit_sales')

In [ ]:
%time x = np.array(trn, dtype=np.float32)
# try also np.float16

In [ ]:
%time x_val = np.array(val, dtype=np.float32)

In [ ]:
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=25,
                          # max_depth=10,
                          min_samples_leaf=10,
                          max_features=0.9,
                          max_samples=0.5,
                         )
%time m.fit(x,y)

In [ ]:
%time print_score(m,x,y,x_val,y_val)
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=25,
                          # max_depth=10,
                          min_samples_leaf=10,
                          max_features=0.9,
                          # bootstrap = True
                          max_samples=0.9,
                         )
%time m.fit(x,y)

In [ ]:
%time print_score(m,x,y,x_val,y_val)
gc.collect()

In [ ]:
preds = np.stack([tree.predict(x_val)
                 for tree in m.estimators_])  # or np.asarray()
w_val = 1 + .25*x_val[:, -6]  # extract `perishable` column

plt.plot([
    # r2_score(y_val, np.mean(preds[:num_trees], axis = 0))
    nwrmsle(np.mean(preds[:num_trees], axis=0), y_val, w_val)
    for num_trees in range(1, 26)
])

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=30,
                          # max_depth=10,
                          min_samples_leaf=5,
                          max_features=0.9,
                          # bootstrap = True
                          max_samples=0.9,
                         )
%time m.fit(x,y)

In [ ]:
%time print_score(m,x,y,x_val,y_val)
gc.collect()